In [115]:
import random
import plotly.graph_objects as go
#random.seed(99)
import pandas as pd
import numpy as np
import scipy
from scipy.stats.stats import pearsonr
import dask.dataframe as dd
import seaborn as sns
import dask.datasets
from dask_sql import Context
import datetime
import time
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

In [116]:
def count_users(name, *list_of_df):
    '''
    Рассчитывает количество уникальных пользователей по каждой рубрике.
    Принимает на вход название сохраняемого файла и таблицы, по которым производится расчет.
    
    '''
    df_all = dd.concat(list(list_of_df))
    users_itogo = df_all.groupby('rubricName').user.nunique().compute()
    users_itogo_result = df_all.user.nunique().compute()

    df20_result_cnt = df20_raw.user.nunique().compute()
    df21_result_cnt = df21_raw.user.nunique().compute()
    df22_result_cnt = df22_raw.user.nunique().compute()
    df23_result_cnt = df23_raw.user.nunique().compute()

    print(users_itogo_result, 
    df20_result_cnt,
    df21_result_cnt,
    df22_result_cnt,
    df23_result_cnt,)

    users_itogo.to_excel('{}users_itogo.xlsx'.format(name))

def agg_df(df, year):
    start_dt = datetime.datetime.now()
    print('Start:', start_dt)
    df1 = df[['user', 'rubricName', 'date']]
    
    try:
        pd_df1_req_cnt = pd.read_excel(f'df1_req_cnt_{year}.xlsx')
        pd_df1_user_cnt = pd.read_excel(f'df1_user_cnt_{year}.xlsx')
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_index=True, right_index=True)
    except:
        df1_user_cnt = df1.groupby('rubricName').user.nunique().compute()
        df1_req_cnt = df1.groupby('rubricName').date.count().compute()
        df1_req_cnt.to_excel(f'df1_req_cnt_{year}.xlsx')
        df1_user_cnt.to_excel(f'df1_user_cnt_{year}.xlsx')
        pd_df1_req_cnt = pd.DataFrame(df1_req_cnt)
        pd_df1_user_cnt = pd.DataFrame(df1_user_cnt)
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_on='rubricName', right_on='rubricName')
    
    df1_agg = df1_agg.sort_values('date', ascending=False)
    df1_agg['rubricName'] = df1_agg['rubricName_x']
    df1_agg = df1_agg.set_index('rubricName')[['date', 'user']].fillna(0).astype(int)
    df1_agg.to_excel(f'df1_agg_{year}.xlsx', index=False)
    end_dt = datetime.datetime.now()
    print('End:', end_dt)
    print('Duration:', end_dt-start_dt)
    return df1_agg

def row_number(df, col, sort_col, index, desc=True, filter_df=True, filter_zero=False):
    if col not in df.columns:
        df[col] = ''
    if filter_df:
        rows_to_count = df[df[col]].sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
    elif filter_zero:
        rows_to_count = df[df[col]!=0].sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
    else:
        rows_to_count = df.sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
        df_shape = df.shape
    rows_to_zero = [i for i in df.index if i not in rows_to_count] 
    n = 0
    for r0 in rows_to_zero:
        df.loc[r0, col] = 0
    for r in rows_to_count:
        n += 1
        df.loc[r, col] = n

    return df

def count_ratings(df, index):
# Пользователи
    df_user_max = df.loc[:,['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']].max(axis=1)
    df_user_min = df.loc[:,['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']].min(axis=1)

    df['Рейтинг по пользователям 2020'] = df_user_max == df.loc[:,['Пользователи 2020']].iloc[:,0]
    df['Рейтинг по пользователям 2021'] = df_user_max == df.loc[:,['Пользователи 2021']].iloc[:,0]
    df['Рейтинг по пользователям 2022'] = df_user_max == df.loc[:,['Пользователи 2022']].iloc[:,0]
    df['Рейтинг по пользователям 2023'] = df_user_max == df.loc[:,['Пользователи 2023']].iloc[:,0]

    df = row_number(df, 'Рейтинг по пользователям 2020', 'Пользователи 2020', index=index)
    df = row_number(df, 'Рейтинг по пользователям 2021', 'Пользователи 2021', index=index)
    df = row_number(df, 'Рейтинг по пользователям 2022', 'Пользователи 2022', index=index)
    df = row_number(df, 'Рейтинг по пользователям 2023', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям, итого', 'Пользователи, итого', filter_df=False, index=index)
    
    
    df['Антирейтинг по пользователям 2020'] = df_user_min == df.loc[:,['Пользователи 2020']].iloc[:,0]
    df['Антирейтинг по пользователям 2021'] = df_user_min == df.loc[:,['Пользователи 2021']].iloc[:,0]
    df['Антирейтинг по пользователям 2022'] = df_user_min == df.loc[:,['Пользователи 2022']].iloc[:,0]
    df['Антирейтинг по пользователям 2023'] = df_user_min == df.loc[:,['Пользователи 2023']].iloc[:,0]

    df = row_number(df, 'Антирейтинг по пользователям 2020', 'Пользователи 2020', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по пользователям 2021', 'Пользователи 2021', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по пользователям 2022', 'Пользователи 2022', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по пользователям 2023', 'Пользователи 2023', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по пользователям, итого', 'Пользователи, итого', desc=False, filter_df=False, index=index)
    
    
    df['Рейтинг постоянного роста\nпользователей 2020-2023'] = ((df['Пользователи 2020'] < df['Пользователи 2021']) & (df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2022'] < df['Пользователи 2023']))
    df['Рейтинг постоянного роста\nпользователей 2021-2023'] = ((df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2022'] < df['Пользователи 2023']))
    df['Рейтинг постоянного роста\nпользователей 2022-2023'] = ((df['Пользователи 2022'] < df['Пользователи 2023']))
    
    df = row_number(df, 'Рейтинг постоянного роста\nпользователей 2020-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nпользователей 2021-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nпользователей 2022-2023', 'Пользователи, итого', index=index)
    
    
    df['Рейтинг постоянного падения\nпользователей 2020-2023'] = ((df['Пользователи 2020'] > df['Пользователи 2021'])  & (df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2022'] > df['Пользователи 2023']))
    df['Рейтинг постоянного падения\nпользователей 2021-2023'] = ((df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2022'] > df['Пользователи 2023']))
    df['Рейтинг постоянного падения\nпользователей 2022-2023'] = ((df['Пользователи 2022'] > df['Пользователи 2023']))
    
    df = row_number(df, 'Рейтинг постоянного падения\nпользователей 2020-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nпользователей 2021-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nпользователей 2022-2023', 'Пользователи, итого', index=index)

    
    df['Рейтинг по пользователям ВНВН'] = ((df['Пользователи 2020'] > df['Пользователи 2021'])  & (df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2022'] > df['Пользователи 2023']))
    df['Рейтинг по пользователям НВНВ'] = ((df['Пользователи 2020'] < df['Пользователи 2021'])  & (df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2022'] < df['Пользователи 2023']))
    df['Рейтинг стабильности\nчисла пользователей'] = abs(df_user_min / df_user_max - 1)

    df = row_number(df, 'Рейтинг по пользователям ВНВН', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям НВНВ', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг стабильности\nчисла пользователей', 'Пользователи, итого', filter_df=False, index=index)
        
# Запросы    
    df_req_max = df.loc[:,['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']].max(axis=1)
    df_req_min = df.loc[:,['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']].min(axis=1)
    df['Рейтинг по запросам 2020'] = df_req_max == df.loc[:,['Запросы 2020']].iloc[:,0]
    df['Рейтинг по запросам 2021'] = df_req_max == df.loc[:,['Запросы 2021']].iloc[:,0]
    df['Рейтинг по запросам 2022'] = df_req_max == df.loc[:,['Запросы 2022']].iloc[:,0]
    df['Рейтинг по запросам 2023'] = df_req_max == df.loc[:,['Запросы 2023']].iloc[:,0]
    
    df = row_number(df, 'Рейтинг по запросам 2020', 'Запросы 2020', index=index)
    df = row_number(df, 'Рейтинг по запросам 2021', 'Запросы 2021', index=index)
    df = row_number(df, 'Рейтинг по запросам 2022', 'Запросы 2022', index=index)
    df = row_number(df, 'Рейтинг по запросам 2023', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам, итого', 'Запросы, итого', filter_df=False, index=index)

    
    df['Антирейтинг по запросам 2020'] = df_req_min == df.loc[:,['Запросы 2020']].iloc[:,0]
    df['Антирейтинг по запросам 2021'] = df_req_min == df.loc[:,['Запросы 2021']].iloc[:,0]
    df['Антирейтинг по запросам 2022'] = df_req_min == df.loc[:,['Запросы 2022']].iloc[:,0]
    df['Антирейтинг по запросам 2023'] = df_req_min == df.loc[:,['Запросы 2023']].iloc[:,0]
    
    df = row_number(df, 'Антирейтинг по запросам 2020', 'Запросы 2020', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по запросам 2021', 'Запросы 2021', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по запросам 2022', 'Запросы 2022', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по запросам 2023', 'Запросы 2023', desc=False, index=index)
    df = row_number(df, 'Антирейтинг по запросам, итого', 'Запросы, итого', desc=False, filter_df=False, index=index)

    
    df['Рейтинг постоянного роста\nзапросов 2020-2023'] = ((df['Запросы 2020'] < df['Запросы 2021']) & (df['Запросы 2021'] < df['Запросы 2022']) & (df['Запросы 2022'] < df['Запросы 2023']))
    df['Рейтинг постоянного роста\nзапросов 2021-2023'] = ((df['Запросы 2021'] < df['Запросы 2022']) & (df['Запросы 2022'] < df['Запросы 2023']))
    df['Рейтинг постоянного роста\nзапросов 2022-2023'] = ((df['Запросы 2022'] < df['Запросы 2023']))
    
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов 2020-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов 2021-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов 2022-2023', 'Запросы, итого', index=index)
    
    
    df['Рейтинг постоянного падения\nзапросов 2020-2023'] = ((df['Запросы 2020'] > df['Запросы 2021'])  & (df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2022'] > df['Запросы 2023']))
    df['Рейтинг постоянного падения\nзапросов 2021-2023'] = ((df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2022'] > df['Запросы 2023']))
    df['Рейтинг постоянного падения\nзапросов 2022-2023'] = ((df['Запросы 2022'] > df['Запросы 2023']))
    
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов 2020-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов 2021-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов 2022-2023', 'Запросы, итого', index=index)
    
    
    df['Рейтинг по запросам ВНВН'] = ((df['Запросы 2020'] > df['Запросы 2021']) & (df['Запросы 2021'] < df['Запросы 2022'])& (df['Запросы 2022'] > df['Запросы 2023']))
    df['Рейтинг по запросам НВНВ'] = ((df['Запросы 2020'] < df['Запросы 2021'])  & (df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2022'] < df['Запросы 2023']))
    df['Рейтинг стабильности\nчисла запросов'] = abs(df_req_min / df_req_max - 1)
    
    df = row_number(df, 'Рейтинг по запросам ВНВН', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам НВНВ', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг стабильности\nчисла запросов', 'Запросы, итого', filter_df=False, index=index)
    
# Запросов на пользователя
    df_req_user_max = df.loc[:,['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']].max(axis=1)
    df_req_user_min = df.loc[:,['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']].min(axis=1)

    df['Рейтинг запросов на пользователя 2020'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2020']].iloc[:,0]
    df['Рейтинг запросов на пользователя 2021'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2021']].iloc[:,0]
    df['Рейтинг запросов на пользователя 2022'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2022']].iloc[:,0]
    df['Рейтинг запросов на пользователя 2023'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2023']].iloc[:,0]

    df = row_number(df, 'Рейтинг запросов на пользователя 2020', 'Запросов на пользователя 2020', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя 2021', 'Запросов на пользователя 2021', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя 2022', 'Запросов на пользователя 2022', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя 2023', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя, итого', 'Запросов на пользователя, итого', filter_df=False, index=index)


    df['Антирейтинг запросов на пользователя 2020'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2020']].iloc[:,0]
    df['Антирейтинг запросов на пользователя 2021'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2021']].iloc[:,0]
    df['Антирейтинг запросов на пользователя 2022'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2022']].iloc[:,0]
    df['Антирейтинг запросов на пользователя 2023'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2023']].iloc[:,0]

    df = row_number(df, 'Антирейтинг запросов на пользователя 2020', 'Запросов на пользователя 2020', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя 2021', 'Запросов на пользователя 2021', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя 2022', 'Запросов на пользователя 2022', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя 2023', 'Запросов на пользователя 2023', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя, итого', 'Запросов на пользователя, итого', desc=False, filter_df=False, index=index)


    df['Рейтинг постоянного роста\nзапросов на пользователя 2020-2023'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2021']) & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного роста\nзапросов на пользователя 2021-2023'] = ((df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного роста\nзапросов на пользователя 2022-2023'] = ((df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))

    df = row_number(df, 'Рейтинг постоянного роста\nзапросов на пользователя 2020-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов на пользователя 2021-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов на пользователя 2022-2023', 'Запросов на пользователя, итого', index=index)


    df['Рейтинг постоянного падения\nзапросов на пользователя 2020-2023'] = ((df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного падения\nзапросов на пользователя 2021-2023'] = ((df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного падения\nзапросов на пользователя 2022-2023'] = ((df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))

    df = row_number(df, 'Рейтинг постоянного падения\nзапросов на пользователя 2020-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов на пользователя 2021-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов на пользователя 2022-2023', 'Запросов на пользователя, итого', index=index)


    df['Рейтинг запросов на пользователя ВНВН'] = ((df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя НВНВ'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))
    df['Рейтинг стабильности\nчисла запросов на пользователя'] = abs(df_user_min / df_user_max - 1)

    df = row_number(df, 'Рейтинг запросов на пользователя ВНВН', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя НВНВ', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг стабильности\nчисла запросов на пользователя', 'Запросов на пользователя, итого', filter_df=False, index=index)
    
#Рейтинг схожести динамики
    x = df[['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']]
    y = df[['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']]
    df = df.fillna(0)
    for row in df.index:
        df.loc[row, 'Антирейтинг схожести динамики\nпользователей и запросов'] = scipy.stats.pearsonr(x.loc[row], y.loc[row])[0]
    df['Рейтинг схожести динамики\nпользователей и запросов'] = df['Антирейтинг схожести динамики\nпользователей и запросов']
    df = row_number(df, 'Рейтинг схожести динамики\nпользователей и запросов', 'Запросы, итого', filter_df=False, index=index)
    df = row_number(df, 'Антирейтинг схожести динамики\nпользователей и запросов', 'Запросы, итого', desc=False, filter_df=False, index=index)
    
    
    x = df[['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']]
    y = df[['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']]
    df = df.fillna(0)
    for row in df.index:
        df.loc[row, 'Антирейтинг схожести динамики\nпользователей и запросов на пользователя'] = scipy.stats.pearsonr(x.loc[row], y.loc[row])[0]
    df['Рейтинг схожести динамики\nпользователей и запросов на пользователя'] = df['Антирейтинг схожести динамики\nпользователей и запросов на пользователя']
    df = row_number(df, 'Рейтинг схожести динамики\nпользователей и запросов на пользователя', 'Запросов на пользователя, итого', filter_df=False, index=index)
    df = row_number(df, 'Антирейтинг схожести динамики\nпользователей и запросов на пользователя', 'Запросов на пользователя, итого', desc=False, filter_df=False, index=index)
    
        
    x = df[['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']]
    y = df[['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']]
    df = df.fillna(0)
    for row in df.index:
        df.loc[row, 'Рейтинг схожести динамики\nзапросов и запросов на пользователя'] = scipy.stats.pearsonr(x.loc[row], y.loc[row])[0]
    df['Антирейтинг схожести динамики\nзапросов и запросов на пользователя'] = df['Рейтинг схожести динамики\nзапросов и запросов на пользователя']
    df = row_number(df, 'Рейтинг схожести динамики\nзапросов и запросов на пользователя', 'Запросов на пользователя, итого', filter_df=False, index=index)
    df = row_number(df, 'Антирейтинг схожести динамики\nзапросов и запросов на пользователя', 'Запросов на пользователя, итого', desc=False, filter_df=False, index=index)
    
    df = df.drop(['№', 'Пользователи 2020', 'Пользователи 2021',
           'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
           'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
           'Запросы 2023', 'Запросы, итого', 'Запросы, %',
           'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
           'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
           'Запросов на пользователя, итого'], axis=1).set_index('Рубрика')
    
    print('Рейтинги построены')
    return df

def horizontal_optimization(table_to_optimize, index):
    new_svod = table_to_optimize
    test_svod = new_svod.copy()
    index_set = set(new_svod.index)
    cnt = 0
    total_sum = 9999
    opt_stats = 0
    iter_stats = 0
    opt_stats2 = 0
    iter_stats2 = 0
    while cnt < 10000 and len(index_set)> 0:
        j = index_set.pop()
        cnt += 1
        cols = [i for i in test_svod.loc[j][test_svod.loc[j] > 0].sort_values(ascending=False).index]

        for i in cols:  
            iter_stats += 1
            cnt_horizontal = test_svod.loc[j].drop('cnt_horizontal').replace(range(1, index+1), 1).sum()
            cnt_vertical = test_svod[i].replace(range(1, index+1), 1).sum()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0
                opt_stats += 1
                
        cols_set = set(cols)
        cnt2 = 0
        while cols_set:
            iter_stats2 += 1
            i = cols_set.pop()
            cnt_horizontal = test_svod.loc[j].drop('cnt_horizontal').replace(range(1, index+1), 1).sum()
            cnt_vertical = test_svod[i].replace(range(1, index+1), 1).sum()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0
                opt_stats2 += 1
        total_sum = test_svod.sum(axis=1).sum()

    test_svod['cnt_horizontal'] = test_svod.drop('cnt_horizontal', axis=1).replace(range(1, index+1), 1).sum(axis=1)
    test_svod = test_svod.sort_values('cnt_horizontal', ascending=False)
    print(opt_stats, iter_stats, opt_stats/iter_stats)
    print(opt_stats2, iter_stats2, opt_stats2/iter_stats2)
    return test_svod

def vertical_optimization(table_to_optimize, original_table, optimization_level, index):
    df = table_to_optimize.copy()
    new_svod = original_table.copy()
    for i in range(optimization_level):
        opt_stats = 0
        iter_stats = 0
        print('Вертикальная оптимизация шаг {} из {}'.format(i+1, optimization_level))
        for cat_to_optimize in df.index:
            iter_stats += 1
            # сейчас есть значение, превращаем в ноль
            col = df.drop('cnt_horizontal', axis=1).loc[cat_to_optimize].sort_values(ascending=False).index[0]

            # сейчас 0 возвращаем значение из бэкапа
            try:
                cat_to_reset = (
                     df
                     .loc[df.index != cat_to_optimize]
                     .loc[(new_svod.sort_index().loc[:, col] != df.sort_index().loc[:, col]) & (df[col] == 0)]
                     .loc[:,[col, 'cnt_horizontal']]
                     .sort_values([col, 'cnt_horizontal'], ascending=[False, True])
                     .index[0]
                )
                if df.loc[cat_to_reset, 'cnt_horizontal'] + 1 < df.loc[cat_to_optimize, 'cnt_horizontal']:
                    df.loc[cat_to_optimize, col] = 0
                    df.loc[cat_to_reset, col] = new_svod.loc[cat_to_reset, col]
                    df['cnt_horizontal'] = df.drop('cnt_horizontal', axis=1).replace(range(1, index+1), 1).sum(axis=1)
                    opt_stats += 1
            except:
                pass
        print(opt_stats, iter_stats, opt_stats/iter_stats)
    return df

def optimize_df(df_to_optimize, index, cut_level, calc_cut_level=False, optimization_level=10):
    original_svod = df_to_optimize.copy().iloc[:cut_level]
    print(original_svod.shape)
    original_svod_sorted = (
        original_svod
        .fillna(0)
        .set_index('Рубрика')
    ).sort_values('Рубрика')
    original_svod_sorted = original_svod.loc[[not i.isnumeric() for i in original_svod['Рубрика']]]
    
    original_svod_counted = count_ratings(original_svod_sorted, index=index)

    original_svod_counted['cnt_horizontal'] = original_svod_counted.replace(range(1, index+1), 1).sum(axis=1)
    original_svod_counted = original_svod_counted.sort_values('cnt_horizontal', ascending=False)

    hor_optimized_svod = horizontal_optimization(original_svod_counted, index=index)
    vert_optimized_svod = vertical_optimization(hor_optimized_svod, original_svod_counted, optimization_level, index=index)
    
    optimized_svod = vert_optimized_svod.reset_index()
    optimized_svod['В рейтингах, раз'] = optimized_svod['cnt_horizontal']
    optimized_svod = optimized_svod.drop('cnt_horizontal', axis=1).sort_index()

    df = (pd.DataFrame(original_svod_counted.value_counts('cnt_horizontal').rename('Оригинальный файл'))
     .merge(pd.DataFrame(hor_optimized_svod.value_counts('cnt_horizontal').rename('Горизонтальная оптимизация')), left_index=True, right_index=True, how='outer')
     .merge(pd.DataFrame(vert_optimized_svod.value_counts('cnt_horizontal').rename('Вертикальная оптимизация')), left_index=True, right_index=True, how='outer')
     .fillna(0).astype(int))

    display(df)
    
    if calc_cut_level:
        return df
    return optimized_svod

def make_final(original_svod, optimized_svod):
    original_svod_sorted = (
        original_svod
        .fillna(0)
        .set_index('Рубрика')
    ).sort_values('Рубрика')
    original_svod_sorted = original_svod.loc[[not i.isnumeric() for i in original_svod['Рубрика']]]
    
    final_svod = original_svod_sorted.copy().reset_index()

    final_svod = final_svod[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
           'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
           'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
           'Запросы 2023', 'Запросы, итого', 'Запросы, %',
           'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
           'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
           'Запросов на пользователя, итого']].sort_values('№')

    final_svod = final_svod.merge(optimized_svod, left_on='Рубрика', right_on='Рубрика')
    final_svod = final_svod.sort_index().sort_values('№')
    for i in optimized_svod.drop(['Рубрика', 'В рейтингах, раз'], axis=1).columns:
        final_svod = row_number(final_svod, i, i, desc=False, filter_df=False, index=15, filter_zero=True)
    final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
    final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)
    final_svod['Вне рейтингов'] = final_svod['В рейтингах, раз'].fillna(0) == 0
    final_svod = row_number(final_svod, 'Вне рейтингов', 'Запросы, итого', desc=True, filter_df=False, index=15, filter_zero=True)

    return final_svod
    
def rename(df1_req_cnt, df1_user_cnt):
    pd_df1_req_cnt = pd.DataFrame(df1_req_cnt)
    pd_df1_user_cnt = pd.DataFrame(df1_user_cnt)
    df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_on='rubricName', right_on='rubricName')
    df1_agg = df1_agg.sort_values('date', ascending=False)
    #df1_agg['rubricName'] = df1_agg['rubricName_x']
    df1_agg = df1_agg.set_index('rubricName')[['date', 'user']].fillna(0).astype(int)
    return df1_agg

#Стартуем с 450 до 750 по 50 пока кол-во 4 не станет = 0
#От найденного значения идем вниз по 10, пока кол-во 4 не станет > 0

def find_cut_level(svod, index, start_cut=400):
    cut_level_num = start_cut
    df = optimize_df(svod, index=index, cut_level=cut_level_num, calc_cut_level=True) 
    while df[df.index > 3]['Вертикальная оптимизация'].sum() > 0:
        cut_level_num += 50
        df = optimize_df(svod, index=index, cut_level=cut_level_num, calc_cut_level=True) 
        
    while df.iloc[3:, 2].sum() == 0 and df.loc[0][2] > 14:
        cut_level_num -= 10
        df = optimize_df(svod, index=index, cut_level=cut_level_num, calc_cut_level=True) 
        
        
    
def show_time(func):
    def inner():
        start_dt = datetime.datetime.now()
        func()
        end_dt = datetime.datetime.now()
        print('Start:'+' '*4, str(start_dt)[:-7])
        print('End:'+' '*6, str(end_dt)[:-7])
        print('Duration:'+' '*1, str(end_dt-start_dt)[:-7])
    return inner


# Москва

In [ ]:
df20_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2020_969792.csv", dtype={'fl': 'object'})
df20_agg = agg_df(df20_raw, 2020)
df21_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2021_970748.csv", dtype={'fl': 'object'})
df21_agg = agg_df(df21_raw, 2021)
df22_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2022_970753.csv", dtype={'fl': 'object'})
df22_agg = agg_df(df22_raw, 2022)
df23_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2023_970754.csv", dtype={'fl': 'object'})
df23_agg = agg_df(df23_raw, 2023)

In [198]:
moscow_svod = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Backups\Мастер_таблица_2ГИС_Москва_230315_1930.xlsx")

In [ ]:
#Стартуем с 450 до 750 по 50 пока кол-во 4 не станет = 0
#От найденного значения идем вниз по 10, пока кол-во 4 не станет > 0

for cut_level_num in range(400, 900, 50):
    original_svod_filtered_optimized, df = optimize_df(moscow_svod, cut_level=cut_level_num, calc_cut_level=True) 
    if df.iloc[4:, 2].sum() == 0:
        break

for cut_level_num2 in range(cut_level_num, 400, -10):
    original_svod_filtered_optimized, df = optimize_df(moscow_svod, cut_level=cut_level_num2, calc_cut_level=True) 
    if df.iloc[4:, 2].sum() > 0 and df.iloc[0, 2] >= 15:
        break

### cut_level Moscow = 430, rating_range = 50

In [234]:
moscow_svod_optimized = optimize_df(moscow_svod, cut_level=430) 

(430, 52)
Рейтинги построены
Вертикальная оптимизация шаг 1 из 10
Вертикальная оптимизация шаг 2 из 10
Вертикальная оптимизация шаг 3 из 10
Вертикальная оптимизация шаг 4 из 10
Вертикальная оптимизация шаг 5 из 10
Вертикальная оптимизация шаг 6 из 10
Вертикальная оптимизация шаг 7 из 10
Вертикальная оптимизация шаг 8 из 10
Вертикальная оптимизация шаг 9 из 10
Вертикальная оптимизация шаг 10 из 10


,Оригинальный файл,Горизонтальная оптимизация,Вертикальная оптимизация
cnt_horizontal,,,
0.0,66,66,66
1.0,37,297,202
2.0,67,21,105
3.0,40,17,56
4.0,86,10,0
5.0,32,2,0
6.0,34,2,0
7.0,9,2,0
8.0,7,4,0


In [249]:
moscow_svod_final = make_final(moscow_svod, moscow_svod_optimized)

C:\Users\Mamed\AppData\Local\Temp\ipykernel_23440\1433869188.py:347: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
C:\Users\Mamed\AppData\Local\Temp\ipykernel_23440\1433869188.py:348: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)


In [250]:
moscow_svod_final.to_excel('moscow_svod_final.xlsx')

# Москва запросы и телефоны

In [287]:
df = dd.read_csv(r"D:\файлы_2гис_2023\ТАШКЕНТ_2020_974993.csv", dtype={'fl': 'object'}).head(5)

df = dd.read_csv(r"D:\файлы_2гис_2023\ТАШКЕНТ_2020_974993.csv", dtype={'fl': 'object'})

df = df[['rubricName','time','date','tx']]

df['time'] = df['time'].apply(lambda x: x.split(' ')[0], meta=('time', 'object'))

In [288]:
dd.read_csv(r"D:\файлы_2гис_2023\ТАШКЕНТ_2020_974993.csv", dtype={'fl': 'object'}).columns

Index(['user', 'proj', 'rubricName', 'rubricID', 'time', 'date', 'org',
       'branch', 'prod', 'tx', 'fl', 'devmod', 'lat', 'lon'],
      dtype='object')

In [ ]:
rubrics = pd.read_excel('final_svod_master.xlsx')[['Рубрика']].iloc[:15]

#df_all = dd.read_parquet('df_all')
#df_filtered = df_all.merge(rubrics, left_on='rubricName', right_on='Рубрика')

#df_filtered.to_parquet('df_filtered')

df_filtered = dd.read_parquet('df_filtered')

In [1]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

In [5]:
spark

In [ ]:
from pyspark.sql.functions import col, udf, coalesce, lit, pandas_udf
from pyspark.sql.types import StringType

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

@udf(StringType()) 
def normalize_word(word):
    return ' '.join(sorted([morph.normal_forms(i)[0] for i in word.split(' ')]))

In [ ]:
df = spark.read.parquet(r"C:\Users\Mamed\_Notebooks\ST\df_filtered")#.withColumn('new_tx',normalize_word(coalesce(col('tx'), lit(''))))

df.createOrReplaceTempView('df')

grouped_df = spark.sql('''

select
    rubricName
    ,year
    ,tx
    ,cnt
    ,row_number() over(partition by rubricName ,year order by cnt desc) rn
from (
    select
        rubricName
        ,year(date) year
        ,lower(tx) tx
        ,count(*) cnt
    from df
    group by 
        rubricName
        ,year(date)
        ,lower(tx)
        ) t
''')#.where('rn < 16')

In [192]:
grouped_df.write.mode('overwrite').parquet('df_tx_grouped')

In [193]:
df_tx_grouped_normalize = spark.read.parquet("df_tx_grouped").withColumn('new_tx',normalize_word(coalesce(col('tx'), lit(''))))

In [194]:
df_tx_grouped_normalize.write.mode('overwrite').parquet('df_tx_grouped_normalize')

In [168]:
spark.read.parquet(r"df_tx_grouped").count()

1480320

In [200]:
spark.read.parquet(r"df_tx_grouped_normalize").show()

+----------+----+--------------------+-----+---+--------------------+
|rubricName|year|                  tx|  cnt| rn|              new_tx|
+----------+----+--------------------+-----+---+--------------------+
|     Банки|2022|            сбербанк|75356|  1|            сбербанк|
|     Банки|2022|                null|40327|  2|                    |
|     Банки|2022|  сбербанк отделения|33827|  3|  отделение сбербанк|
|     Банки|2022|                 втб|32847|  4|                 втб|
|     Банки|2022|альфа банк отделения|30460|  5|альфа банк отделение|
|     Банки|2022|          совкомбанк|12579|  6|          совкомбанк|
|     Банки|2022|            банк втб|12356|  7|            банк втб|
|     Банки|2022|               банки|11585|  8|                банк|
|     Банки|2022|газпромбанк отдел...| 9986|  9|газпромбанк отдел...|
|     Банки|2022|райффайзенбанк от...| 8162| 10|отделение райффай...|
|     Банки|2022|          почта банк| 8058| 11|          банк почта|
|     Банки|2022|   

In [197]:
df_tx = spark.read.parquet(r"df_tx_grouped_normalize")

In [198]:
df_tx.createOrReplaceTempView('df_tx')

In [201]:
df_tx_grouped_normalize_final = spark.sql('''

select
    rubricName
    ,year
    ,new_tx
    ,cnt
    ,row_number() over(partition by rubricName ,year order by cnt desc) rn
from (
    select
        rubricName
        ,year
        ,new_tx
        ,sum(cnt) cnt
    from df_tx
    group by 
        rubricName
        ,year
        ,new_tx
        ) t
''').where('rn < 16')

In [202]:
df_tx_grouped_normalize_final.write.mode('overwrite').parquet('df_tx_grouped_normalize_final')

In [204]:
spark.read.parquet('df_tx_grouped_normalize_final').count()

1135

In [205]:
pdf = spark.read.parquet('df_tx_grouped_normalize_final').toPandas()

In [208]:
pdf.to_excel('pdf.xlsx')

In [212]:
grouped_df_devmod = spark.sql('''

select
    rubricName
    ,year
    ,devmod
    ,cnt
    ,row_number() over(partition by rubricName ,year order by cnt desc) rn
from (
    select
        rubricName
        ,year(date) year
        ,lower(devmod) devmod
        ,count(*) cnt
    from df
    group by 
        rubricName
        ,year(date)
        ,lower(devmod)
        ) t
''').where('rn < 16')

In [214]:
grouped_df_devmod.write.mode('overwrite').parquet('df_devmod_grouped')

In [215]:
pdf2 = spark.read.parquet('df_devmod_grouped').toPandas()

In [216]:
pdf2.to_excel('pdf2.xlsx')

# СПБ

In [ ]:
spb20_raw = dd.read_csv(r"C:\Users\Mamed\Downloads\СПБ_2020_974783.csv")
spb20_agg = agg_df(spb20_raw, 'spb_2020')

In [ ]:
spb21_raw = dd.read_csv(r"C:\Users\Mamed\Downloads\СПБ_2021_974784.csv")
spb21_agg = agg_df(spb21_raw, 'spb_2021')

# Алматы

In [ ]:
df23_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Алмата_2023_974800.csv", dtype={'fl': 'object'})
df23_agg = agg_df(df23_raw, 'алмата_2023')

In [ ]:
df22_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Алмата_2022_974799.csv", dtype={'fl': 'object'})
df22_agg = agg_df(df22_raw, 'алмата_2022')

In [ ]:
df21_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Алмата_2021_974798.csv", dtype={'fl': 'object'})
df21_agg = agg_df(df21_raw, 'алмата_2021')

In [7]:
df20_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Алмата_2020_974797.csv", dtype={'fl': 'object'})
df20_agg = agg_df(df20_raw, 'алмата_2020')

Start: 2023-03-17 03:34:44.235204
End: 2023-03-17 03:59:45.969968
Duration: 0:25:01.734764


In [7]:
df1_req_cnt_алмата_2020 = pd.read_excel('df1_req_cnt_алмата_2020.xlsx')
df1_req_cnt_алмата_2021 = pd.read_excel('df1_req_cnt_алмата_2021.xlsx')
df1_req_cnt_алмата_2022 = pd.read_excel('df1_req_cnt_алмата_2022.xlsx')
df1_req_cnt_алмата_2023 = pd.read_excel('df1_req_cnt_алмата_2023.xlsx')

df1_user_cnt_алмата_2020 = pd.read_excel('df1_user_cnt_алмата_2020.xlsx')
df1_user_cnt_алмата_2021 = pd.read_excel('df1_user_cnt_алмата_2021.xlsx')
df1_user_cnt_алмата_2022 = pd.read_excel('df1_user_cnt_алмата_2022.xlsx')
df1_user_cnt_алмата_2023 = pd.read_excel('df1_user_cnt_алмата_2023.xlsx')

df1_cnt_алмата_2020 = rename(df1_req_cnt_алмата_2020, df1_user_cnt_алмата_2020)
df1_cnt_алмата_2021 = rename(df1_req_cnt_алмата_2021, df1_user_cnt_алмата_2021)
df1_cnt_алмата_2022 = rename(df1_req_cnt_алмата_2022, df1_user_cnt_алмата_2022)
df1_cnt_алмата_2023 = rename(df1_req_cnt_алмата_2023, df1_user_cnt_алмата_2023)

df1_cnt_алмата_2020_2021 = df1_cnt_алмата_2020.merge(df1_cnt_алмата_2021, how='outer', left_index=True, right_index=True)

df1_cnt_алмата_2022_2023 = df1_cnt_алмата_2022.merge(df1_cnt_алмата_2023, how='outer', left_index=True, right_index=True)

columns1 = ['Запросы 2020','Пользователи 2020','Запросы 2021','Пользователи 2021']

columns2 = ['Запросы 2022','Пользователи 2022','Запросы 2023','Пользователи 2023']

df1_cnt_алмата_2020_2021.columns = columns1

df1_cnt_алмата_2022_2023.columns = columns2

df1_cnt_алмата = df1_cnt_алмата_2020_2021.merge(df1_cnt_алмата_2022_2023, how='outer', left_index=True, right_index=True).reset_index()

df1_cnt_алмата['Рубрика'] = df1_cnt_алмата['rubricName']

df1_cnt_алмата = df1_cnt_алмата.drop('rubricName', axis=1)

In [61]:
count_users_requests('almata', df20_raw, df21_raw, df22_raw, df23_raw)

4781734 1420348 1516714 1801050 2297946


In [8]:
almata_users_itogo = pd.read_excel('almatausers_itogo.xlsx')

almata_svod = df1_cnt_алмата.merge(almata_users_itogo, right_on='rubricName', left_on='Рубрика')

columns = ['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']

almata_svod['Пользователи, итого'] = almata_svod['user']

for i in columns:
    if i not in almata_svod.columns:
        almata_svod[i] = ''

almata_svod = almata_svod[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']]

In [9]:
#almata_svod.to_excel('almata_svod.xlsx')

In [166]:
almata_svod_itogo = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Таблица_2ГИС_Алматы_Рейтинги_230317_1315.xlsx")

In [167]:
almata_svod_itogo = almata_svod_itogo[almata_svod_itogo['№'] != 'Итог'].fillna(0)

In [280]:
#Стартуем с 450 до 750 по 50 пока кол-во 4 не станет = 0
#От найденного значения идем вниз по 10, пока кол-во 4 не станет > 0

def find_cut_level(svod, index=50, start_cut=400):
    for cut_level_num in range(start_cut, 1150, 50):
        original_svod_filtered_optimized, df = optimize_df(svod, index=index, cut_level=cut_level_num, calc_cut_level=True) 
        if df.iloc[4:, 2].sum() == 0:
            break

    for cut_level_num2 in range(cut_level_num-10, start_cut, -10):
        original_svod_filtered_optimized, df = optimize_df(svod, index=index, cut_level=cut_level_num2, calc_cut_level=True) 
        if df.iloc[4:, 2].sum() > 0 and df.iloc[0, 2] >= 15:
            break
            
    return original_svod_filtered_optimized

In [278]:
for cut_level_num2 in range(1050, 900, -10):
        original_svod_filtered_optimized, df = optimize_df(almata_svod_itogo, index=80, cut_level=cut_level_num2, calc_cut_level=True) 
        if df.iloc[4:, 2].sum() > 0:
            break

(1050, 52)
Рейтинги построены
Вертикальная оптимизация шаг 1 из 10
Вертикальная оптимизация шаг 2 из 10
Вертикальная оптимизация шаг 3 из 10
Вертикальная оптимизация шаг 4 из 10
Вертикальная оптимизация шаг 5 из 10
Вертикальная оптимизация шаг 6 из 10
Вертикальная оптимизация шаг 7 из 10
Вертикальная оптимизация шаг 8 из 10
Вертикальная оптимизация шаг 9 из 10
Вертикальная оптимизация шаг 10 из 10


,Оригинальный файл,Горизонтальная оптимизация,Вертикальная оптимизация
cnt_horizontal,,,
0.0,611,611,611
1.0,102,375,291
2.0,102,19,88
3.0,47,17,60
4.0,54,9,0
5.0,30,3,0
6.0,36,6,0
7.0,11,1,0
8.0,9,1,0


(1040, 52)
Рейтинги построены
Вертикальная оптимизация шаг 1 из 10
Вертикальная оптимизация шаг 2 из 10
Вертикальная оптимизация шаг 3 из 10
Вертикальная оптимизация шаг 4 из 10
Вертикальная оптимизация шаг 5 из 10
Вертикальная оптимизация шаг 6 из 10
Вертикальная оптимизация шаг 7 из 10
Вертикальная оптимизация шаг 8 из 10
Вертикальная оптимизация шаг 9 из 10
Вертикальная оптимизация шаг 10 из 10


,Оригинальный файл,Горизонтальная оптимизация,Вертикальная оптимизация
cnt_horizontal,,,
0.0,602,602,602
1.0,102,374,296
2.0,103,21,82
3.0,46,17,59
4.0,53,8,1
5.0,32,2,0
6.0,36,6,0
7.0,10,1,0
8.0,9,1,0


### Алмата cut_level = 1050, rating_range = 80

In [281]:
almata_optimized = find_cut_level(almata_svod_itogo, 80, 1050)

(1050, 52)
Рейтинги построены
Вертикальная оптимизация шаг 1 из 10
Вертикальная оптимизация шаг 2 из 10
Вертикальная оптимизация шаг 3 из 10
Вертикальная оптимизация шаг 4 из 10
Вертикальная оптимизация шаг 5 из 10
Вертикальная оптимизация шаг 6 из 10
Вертикальная оптимизация шаг 7 из 10
Вертикальная оптимизация шаг 8 из 10
Вертикальная оптимизация шаг 9 из 10
Вертикальная оптимизация шаг 10 из 10


,Оригинальный файл,Горизонтальная оптимизация,Вертикальная оптимизация
cnt_horizontal,,,
0.0,611,611,611
1.0,102,375,291
2.0,102,19,88
3.0,47,17,60
4.0,54,9,0
5.0,30,3,0
6.0,36,6,0
7.0,11,1,0
8.0,9,1,0


In [282]:
almata_final = make_final(almata_svod_itogo, almata_optimized)

C:\Users\Mamed\AppData\Local\Temp\ipykernel_23440\1433869188.py:347: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
C:\Users\Mamed\AppData\Local\Temp\ipykernel_23440\1433869188.py:348: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)


In [283]:
almata_final.to_excel('almata_final.xlsx')

# Новосибирск

### данные

In [144]:
#2020
nov_df20_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Новосибирск_2020_974791.csv", dtype={'fl': 'object'})
nov_df20_agg = agg_df(nov_df20_raw, 'новосиб_2020')

Start: 2023-03-17 04:18:08.684168
End: 2023-03-17 04:18:08.855440
Duration: 0:00:00.171272


In [142]:
#2021
nov_df21_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Новосибирск_2021_974792.csv", dtype={'fl': 'object'})
nov_df21_agg = agg_df(nov_df21_raw, 'новосиб_2021')

Start: 2023-03-17 04:18:07.367428
End: 2023-03-17 04:18:07.543124
Duration: 0:00:00.175696


In [143]:
#2022
nov_df22_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Новосибирск_2022_974795.csv", dtype={'fl': 'object'})
nov_df22_agg = agg_df(nov_df22_raw, 'новосиб_2022')

Start: 2023-03-17 04:18:07.859904
End: 2023-03-17 04:18:08.033286
Duration: 0:00:00.173382


In [141]:
#2023 Новосибирск_2023_974796 Новосибирск_2020_974791
nov_df23_raw = dd.read_csv(r"C:\Users\Mamed\Desktop\файлы_2гис_2023\Новосибирск_2023_974796.csv", dtype={'fl': 'object'})
nov_df23_agg = agg_df(nov_df23_raw, 'новосиб_2023')

Start: 2023-03-17 04:18:04.432750
End: 2023-03-17 04:18:04.772263
Duration: 0:00:00.339513


In [132]:
df1_req_cnt_новосиб_2020 = pd.read_excel('df1_req_cnt_новосиб_2020.xlsx')
df1_req_cnt_новосиб_2021 = pd.read_excel('df1_req_cnt_новосиб_2021.xlsx')
df1_req_cnt_новосиб_2022 = pd.read_excel('df1_req_cnt_новосиб_2022.xlsx')
df1_req_cnt_новосиб_2023 = pd.read_excel('df1_req_cnt_новосиб_2023.xlsx')

df1_user_cnt_новосиб_2020 = pd.read_excel('df1_user_cnt_новосиб_2020.xlsx')
df1_user_cnt_новосиб_2021 = pd.read_excel('df1_user_cnt_новосиб_2021.xlsx')
df1_user_cnt_новосиб_2022 = pd.read_excel('df1_user_cnt_новосиб_2022.xlsx')
df1_user_cnt_новосиб_2023 = pd.read_excel('df1_user_cnt_новосиб_2023.xlsx')

df1_cnt_новосиб_2020 = rename(df1_req_cnt_новосиб_2020, df1_user_cnt_новосиб_2020)
df1_cnt_новосиб_2021 = rename(df1_req_cnt_новосиб_2021, df1_user_cnt_новосиб_2021)
df1_cnt_новосиб_2022 = rename(df1_req_cnt_новосиб_2022, df1_user_cnt_новосиб_2022)
df1_cnt_новосиб_2023 = rename(df1_req_cnt_новосиб_2023, df1_user_cnt_новосиб_2023)

df1_cnt_новосиб_2020_2021 = df1_cnt_новосиб_2020.merge(df1_cnt_новосиб_2021, how='outer', left_index=True, right_index=True)

df1_cnt_новосиб_2022_2023 = df1_cnt_новосиб_2022.merge(df1_cnt_новосиб_2023, how='outer', left_index=True, right_index=True)

columns1 = ['Запросы 2020','Пользователи 2020','Запросы 2021','Пользователи 2021']

columns2 = ['Запросы 2022','Пользователи 2022','Запросы 2023','Пользователи 2023']

df1_cnt_новосиб_2020_2021.columns = columns1

df1_cnt_новосиб_2022_2023.columns = columns2

df1_cnt_новосиб = df1_cnt_новосиб_2020_2021.merge(df1_cnt_новосиб_2022_2023, how='outer', left_index=True, right_index=True).reset_index()

df1_cnt_новосиб['Рубрика'] = df1_cnt_новосиб['rubricName']

df1_cnt_новосиб = df1_cnt_новосиб.drop('rubricName', axis=1)

In [145]:
count_users_requests('nov', nov_df20_raw, nov_df21_raw, nov_df22_raw, nov_df23_raw)

2785152 1212936 1067135 1158251 1084689


In [155]:
nov_users_itogo = pd.read_excel('novusers_itogo.xlsx')

In [165]:
novosib_svod = df1_cnt_новосиб.merge(nov_users_itogo, right_on='rubricName', left_on='Рубрика')

In [169]:
columns = ['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']

In [170]:
novosib_svod['Пользователи, итого'] = novosib_svod['user']

In [177]:
for i in columns:
    if i not in novosib_svod.columns:
        novosib_svod[i] = ''

In [179]:
novosib_svod = novosib_svod[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']]

In [182]:
#novosib_svod.to_excel('novosib_svod.xlsx')

### find cut_level Novosib

In [258]:
novosib_optimized_final.columns

Index(['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого', 'Рейтинг по пользователям 2020',
       'Рейтинг по пользователям 2021', 'Рейтинг по пользователям 2022',
       'Рейтинг по пользователям 2023', 'Рейтинг по пользователям, итого',
       'Антирейтинг по пользователям 2020',
       'Антирейтинг по пользователям 2021',
       'Антирейтинг по пользователям 2022',
       'Антирейтинг по пользователям 2023',
       'Антирейтинг по пользователям, итого',
       'Рейтинг постоянного роста\nпользователей 2020-2023',
       'Рейтинг постоянного роста\nпользователей 2021-2023',
       'Рейтинг посто

In [257]:
novosib_svod_itogo.columns

Index(['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого', 'Рейтинг по пользователям, итого',
       'Рейтинг по пользователям 2020', 'Рейтинг по пользователям 2021',
       'Рейтинг по пользователям 2022', 'Рейтинг по пользователям 2023',
       'Антирейтинг по пользователям 2020',
       'Антирейтинг по пользователям 2021',
       'Антирейтинг по пользователям 2022',
       'Антирейтинг по пользователям 2023',
       'Рейтинг постоянного\nроста пользователей',
       'Рейтинг постоянного\nпадения пользователей',
       'Рейтинг по пользователям ВНВН', 'Рейтинг по пользователям НВНВ',
       'Рейт

In [264]:
novosib_svod_itogo = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Backups\Таблица_2ГИС_Новосибирск_Рейтинги_230317_1245.xlsx")

In [265]:
novosib_svod_itogo = novosib_svod_itogo[novosib_svod_itogo['№'] != 'Итог'].fillna(0).iloc[:,:19]

In [266]:
novosib_svod_itogo.head(5)

,№,Рубрика,Пользователи 2020,Пользователи 2021,Пользователи 2022,Пользователи 2023,"Пользователи, итого","Пользователи, %",Запросы 2020,Запросы 2021,Запросы 2022,Запросы 2023,"Запросы, итого","Запросы, %",Запросов на пользователя 2020,Запросов на пользователя 2021,Запросов на пользователя 2022,Запросов на пользователя 2023,"Запросов на пользователя, итого"
0,1,Бани / Сауны,115053,110033,110010,115841,360913,0.938076,780070,1028402,936664,944554,3689690,2.389666,6.780093,9.346305,8.514353,8.153883,10.223212
1,2,Кафе,209550,176380,189021,194712,591546,1.537532,868618,833282,860054,916422,3478376,2.252806,4.145159,4.724357,4.550045,4.706551,5.880145
2,3,Аптеки,216260,166665,180786,168200,569352,1.479846,911330,765507,797133,722125,3196095,2.069984,4.214048,4.593088,4.409263,4.293252,5.613566
3,4,Быстрое питание,211881,153575,178653,187354,581162,1.510542,752529,618650,746929,849372,2967480,1.921919,3.551659,4.028325,4.180893,4.533514,5.106115
4,5,Рестораны,152315,138470,152737,174383,460606,1.197196,656222,679222,726381,904171,2965996,1.920958,4.308322,4.905192,4.755763,5.184972,6.439334


In [205]:
novosib_ratings = count_ratings(novosib_svod_itogo, index=70)

C:\Users\Mamed\Anaconda\lib\site-packages\scipy\stats\stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
C:\Users\Mamed\Anaconda\lib\site-packages\scipy\stats\stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Рейтинги построены


### cut_level Novosib = 500, rating_range = 50

### cut_level Novosib = 750, rating_range = 70

In [267]:
novosib_optimized = optimize_df(novosib_svod_itogo, index=70, cut_level=750)

(750, 19)
Рейтинги построены
3314 4931 0.6720746298925168
378 4931 0.07665787872642466
Вертикальная оптимизация шаг 1 из 10
62 750 0.08266666666666667
Вертикальная оптимизация шаг 2 из 10
46 750 0.06133333333333333
Вертикальная оптимизация шаг 3 из 10
35 750 0.04666666666666667
Вертикальная оптимизация шаг 4 из 10
29 750 0.03866666666666667
Вертикальная оптимизация шаг 5 из 10
19 750 0.025333333333333333
Вертикальная оптимизация шаг 6 из 10
14 750 0.018666666666666668
Вертикальная оптимизация шаг 7 из 10
12 750 0.016
Вертикальная оптимизация шаг 8 из 10
9 750 0.012
Вертикальная оптимизация шаг 9 из 10
5 750 0.006666666666666667
Вертикальная оптимизация шаг 10 из 10
4 750 0.005333333333333333


,Оригинальный файл,Горизонтальная оптимизация,Вертикальная оптимизация
cnt_horizontal,,,
0.0,23,23,23
1.0,56,635,428
2.0,93,30,261
3.0,87,16,38
4.0,102,11,0
5.0,84,6,0
6.0,86,7,0
7.0,40,6,0
8.0,38,4,0


In [268]:
novosib_optimized_final = make_final(novosib_svod_itogo, novosib_optimized)

C:\Users\Mamed\AppData\Local\Temp\ipykernel_3880\2801575057.py:402: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
C:\Users\Mamed\AppData\Local\Temp\ipykernel_3880\2801575057.py:403: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)


In [269]:
novosib_optimized_final.to_excel('novosib_optimized_final.xlsx')

# Минск

In [10]:
def agg_df(df, year):
    start_dt = datetime.datetime.now()
    print('Start:', start_dt)
    df1 = df[['user', 'rubricName', 'date']]
    
    try:
        pd_df1_req_cnt = pd.read_excel(f'df1_req_cnt_{year}.xlsx')
        pd_df1_user_cnt = pd.read_excel(f'df1_user_cnt_{year}.xlsx')
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_index=True, right_index=True)
    except:
        df1_user_cnt = df1.groupby('rubricName').user.nunique().compute()
        df1_req_cnt = df1.groupby('rubricName').date.count().compute()
        df1_req_cnt.to_excel(f'df1_req_cnt_{year}.xlsx')
        df1_user_cnt.to_excel(f'df1_user_cnt_{year}.xlsx')
        pd_df1_req_cnt = pd.DataFrame(df1_req_cnt)
        pd_df1_user_cnt = pd.DataFrame(df1_user_cnt)
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_on='rubricName', right_on='rubricName')
    
    df1_agg = df1_agg.sort_values('date', ascending=False)
    df1_agg['rubricName'] = df1_agg['rubricName_x']
    df1_agg = df1_agg.set_index('rubricName')[['date', 'user']].fillna(0).astype(int)
    df1_agg.to_excel(f'df1_agg_{year}.xlsx', index=False)
    end_dt = datetime.datetime.now()
    print('End:', end_dt)
    print('Duration:', end_dt-start_dt)
    return df1_agg

In [10]:
def agg_df(df, year):
    start_dt = datetime.datetime.now()
    print('Start:', start_dt)
    df1 = df[['user', 'rubricName', 'date']]
    df1_user_cnt = df1.groupby('rubricName').user.nunique().compute()
    df1_req_cnt = df1.groupby('rubricName').date.count().compute()
    df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_on='rubricName', right_on='rubricName')
    
    df1_agg = df1_agg.sort_values('date', ascending=False)
    df1_agg['rubricName'] = df1_agg['rubricName_x']
    df1_agg = df1_agg.set_index('rubricName')[['date', 'user']].fillna(0).astype(int)
    df1_agg.to_excel(f'df1_agg_{year}.xlsx', index=False)
    end_dt = datetime.datetime.now()
    print('End:', end_dt)
    print('Duration:', end_dt-start_dt)
    return df1_agg

In [123]:
file_path = "D:\файлы_2гис_2023\Калининград\Калининград_2020_975659.csv"
year = 2020



In [124]:
count_users_requests(file_path)

,Запросы 2020,Пользователи 2020
npartitions=1,,
,int32,int32
,...,...


In [117]:
def count_users_requests(file_path):
    year = file_path.split('_')[-2]
    requests_name = f'Запросы {year}'
    users_name = f'Пользователи {year}'
    df = dd.read_csv(file_path)[['user', 'rubricName', 'date']]
    cnt_user_grouped = df.groupby('rubricName').user.nunique().to_frame()
    cnt_req_grouped = df.groupby('rubricName').date.count().to_frame()
    user_cnt_year = df.user.nunique().compute()
    print(year, user_cnt_year)
    cnt_user_grouped_year = (cnt_req_grouped
                  .merge(cnt_user_grouped, left_on='rubricName', right_on='rubricName')
                  .rename(columns={'date':requests_name, 'user':users_name})
                  .sort_values(requests_name, ascending=False)
                  .fillna(0)
                  .astype(int)
                  )
    return cnt_user_grouped_year


def count_total_users(*files):
    df_list = [dd.read_csv(file_path)[['rubricName', 'user']] for file_path in files]
    df_all = dd.concat(df_list)
    cnt_user_grouped_total = df_all.groupby('rubricName').user.nunique().to_frame().rename(columns={'user':'Пользователи, итого'})
    cnt_user_total = df_all.user.nunique().compute()
    print(cnt_user_total)
    return cnt_user_grouped_total

@show_time
def make_base_counted_file(*files):
    df = count_users_requests(files[0])
    df.to_excel(f'{files[0]}_count_users_requests.xlsx')
    for file in files[1:]:
        df = df.merge(count_users_requests(file), left_on='rubricName', right_on='rubricName')
        df.to_excel(f'{file}_count_users_requests.xlsx')
        
    df = df.merge(count_total_users(files), left_on='rubricName', right_on='rubricName')
    df.to_excel('{}_count_total_users.xlsx'.format(file.split("\\")[-2]))
    return df


In [105]:
x = make_base_counted_file(
"D:\файлы_2гис_2023\Калининград\Калининград_2020_975659.csv",
"D:\файлы_2гис_2023\Калининград\Калининград_2021_975670.csv",
"D:\файлы_2гис_2023\Калининград\Калининград_2022_975672.csv",
"D:\файлы_2гис_2023\Калининград\Калининград_2023_975679.csv",

)

2020 dd.Scalar<series-..., dtype=int64>
2021 dd.Scalar<series-..., dtype=int64>
2022 dd.Scalar<series-..., dtype=int64>
2023 dd.Scalar<series-..., dtype=int64>
dd.Scalar<series-..., dtype=int64>


In [115]:
columns = ['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']

In [132]:
"D:\файлы_2гис_2023\Калининград\Калининград_2020_975659.csv".split('_')[-2]

'2020'

In [114]:
df_counted

,requests,user
npartitions=1,,
,int32,int32
,...,...


In [ ]:
"D:\файлы_2гис_2023\Калининград\Калининград_2020_975659.csv"
"D:\файлы_2гис_2023\Калининград\Калининград_2021_975670.csv"
"D:\файлы_2гис_2023\Калининград\Калининград_2022_975672.csv"
"D:\файлы_2гис_2023\Калининград\Калининград_2023_975679.csv"

In [20]:
df22_raw = dd.read_csv(r"D:\файлы_2гис_2023\Минск\Минск_2022_975696.csv", dtype={'fl': 'object'})
df22_agg = agg_df(df22_raw, 'Минск_2022')

Start: 2023-03-21 21:44:33.470554
End: 2023-03-21 21:44:33.630872
Duration: 0:00:00.160318


In [21]:
df23_raw = dd.read_csv(r"D:\файлы_2гис_2023\Минск\Минск_2023_975719.csv", dtype={'fl': 'object'})
df23_agg = agg_df(df23_raw, 'Минск_2023')

Start: 2023-03-21 21:44:33.723429
End: 2023-03-21 21:44:33.857545
Duration: 0:00:00.134116


In [22]:
df1_req_cnt_минск_2022 = pd.read_excel('df1_req_cnt_минск_2022.xlsx')
df1_req_cnt_минск_2023 = pd.read_excel('df1_req_cnt_минск_2023.xlsx')

df1_user_cnt_минск_2022 = pd.read_excel('df1_user_cnt_минск_2022.xlsx')
df1_user_cnt_минск_2023 = pd.read_excel('df1_user_cnt_минск_2023.xlsx')

df1_cnt_минск_2022 = rename(df1_req_cnt_минск_2022, df1_user_cnt_минск_2022)
df1_cnt_минск_2023 = rename(df1_req_cnt_минск_2023, df1_user_cnt_минск_2023)

df1_cnt_минск_2022_2023 = df1_cnt_минск_2022.merge(df1_cnt_минск_2023, how='outer', left_index=True, right_index=True)

columns2 = ['Запросы 2022','Пользователи 2022','Запросы 2023','Пользователи 2023']

df1_cnt_минск_2022_2023.columns = columns2

df1_cnt_минск = df1_cnt_минск_2022_2023.reset_index()

df1_cnt_минск['Рубрика'] = df1_cnt_минск['rubricName']

df1_cnt_минск = df1_cnt_минск.drop('rubricName', axis=1)

In [29]:
df_all = dd.concat(list([df22_raw, df23_raw]))
users_itogo = df_all.groupby('rubricName').user.nunique().compute()
users_itogo_result = df_all.user.nunique().compute()
df22_result_cnt = df22_raw.user.nunique().compute()
df23_result_cnt = df23_raw.user.nunique().compute()
users_itogo.to_excel('{}users_itogo.xlsx'.format('minsk'))

In [30]:
minsk_users_itogo = pd.read_excel('minskusers_itogo.xlsx')

minsk_svod = df1_cnt_минск.merge(minsk_users_itogo, right_on='rubricName', left_on='Рубрика')

columns = ['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']

minsk_svod['Пользователи, итого'] = minsk_svod['user']

for i in columns:
    if i not in minsk_svod.columns:
        minsk_svod[i] = ''

minsk_svod = minsk_svod[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
       'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
       'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
       'Запросы 2023', 'Запросы, итого', 'Запросы, %',
       'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
       'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
       'Запросов на пользователя, итого']]

In [31]:
minsk_svod.to_excel('minsk_svod.xlsx')